In [1]:
import pandas as pd
import numpy as np
import operator
from tqdm import tqdm


In [2]:
ROOT_DIR = "C:/Users/evbruh/Downloads/rsna-intracranial-hemorrhage-detection/"
ALL_DF_PATH = ROOT_DIR + 'all_df.p'
TRAIN_DF_PATH = ROOT_DIR + 'train_df.p'
VAL_DF_PATH = ROOT_DIR + 'val_df.p'
batch_size = 32


In [3]:
all_df = pd.read_pickle(ALL_DF_PATH)
all_df.head(10)

ID Label                                             \
Subtype                none epidural intraparenchymal intraventricular   
0        ID_fd3c840c9     0        0                1                0   
1        ID_07389b816     1        0                0                0   
2        ID_e3ac06794     0        0                0                0   
3        ID_b6200a7a1     1        0                0                0   
4        ID_e192b2995     1        0                0                0   
5        ID_4ae953e81     0        0                0                0   
6        ID_3b713f439     1        0                0                0   
7        ID_594f2d851     1        0                0                0   
8        ID_4b0d1e312     1        0                0                0   
9        ID_4076b3899     1        0                0                0   

                               \
Subtype subarachnoid subdural   
0                  1        0   
1                  0        0   
2                  1        0   
3                  0        0   
4                  0        0   
5                  0        1   
6                  0        0   
7                  0        0   
8                  0        0   
9                  0        0   

                                                  filepath  
Subtype                                                     
0        C:/Users/evbruh/Downloads/rsna-intracranial-he...  
1        C:/Users/evbruh/Downloads/rsna-intracranial-he...  
2        C:/Users/evbruh/Downloads/rsna-intracranial-he...  
3        C:/Users/evbruh/Downloads/rsna-intracranial-he...  
4        C:/Users/evbruh/Downloads/rsna-intracranial-he...  
5        C:/Users/evbruh/Downloads/rsna-intracranial-he...  
6        C:/Users/evbruh/Downloads/rsna-intracranial-he...  
7        C:/Users/evbruh/Downloads/rsna-intracranial-he...  
8        C:/Users/evbruh/Downloads/rsna-intracranial-he...  
9        C:/Users/evbruh/Downloads/rsna-intracranial-he...

In [4]:
n_folds = 5
fold_size = int(all_df.shape[0]*(1-1/n_folds))
for n_fold in tqdm(range(n_folds)):
    
    s_ind = int(n_fold*(1/n_folds)*all_df.shape[0])
    
    train_df_inds = np.arange(s_ind, s_ind + fold_size)%all_df.shape[0]
    train_df = all_df.loc[train_df_inds]
    train_df = train_df[:train_df.shape[0]-train_df.shape[0]%batch_size]
    
    val_df_inds = np.delete(np.arange(all_df.shape[0]), train_df_inds)
    val_df = all_df.loc[val_df_inds]
    val_df = val_df[:val_df.shape[0]-val_df.shape[0]%batch_size]
    
    labels = list(train_df.Label.columns)
    original_counts = dict(train_df.Label.sum(axis=0))
    major_class, major_class_count = max(original_counts.items(), key=operator.itemgetter(1))
    minor_classes_counts = {x:original_counts[x] for x in labels if x is not major_class}
    minor_classes_count = sum(minor_classes_counts.values())
    ratio = major_class_count/minor_classes_count
    new_minor_classes_counts = {x:int(original_counts[x]*ratio) for x in minor_classes_counts}
    new_counts = {**{major_class:major_class_count}, **new_minor_classes_counts}
    class_dfs = {x:train_df[train_df.Label[x]==1] for x in labels}
    for x in labels:
        if x is major_class:
            class_dfs[x] = class_dfs[x]
        else:
            class_dfs[x] = class_dfs[x].sample(new_counts[x], replace=True)
    
    train_df = pd.concat([class_dfs[x] for x in labels])
    train_df.index = np.arange(train_df.shape[0]).astype('int64')
    train_df = train_df.reindex(np.random.permutation(train_df.index))
    train_df.index = np.arange(train_df.shape[0]).astype('int64')
    
    train_df.to_pickle(TRAIN_DF_PATH[:-2] + '_fold_{}'.format(n_fold) + TRAIN_DF_PATH[-2:])
    val_df.to_pickle(VAL_DF_PATH[:-2] + '_fold_{}'.format(n_fold) + VAL_DF_PATH[-2:])

    

100%|██████████| 5/5 [00:17<00:00,  3.43s/it]


In [9]:
TRAIN_DF_PATH[:-2] + '_fold_{}'.format(n_fold) + TRAIN_DF_PATH[-2:]

'C:/Users/evbruh/Downloads/rsna-intracranial-hemorrhage-detection/train_df_fold_4.p'

In [10]:
VAL_DF_PATH[:-2] + '_fold_{}'.format(n_fold) + VAL_DF_PATH[-2:]

'C:/Users/evbruh/Downloads/rsna-intracranial-hemorrhage-detection/val_df_fold_4.p'

In [7]:
train_df_inds, train_df_inds.shape

(array([539191, 539192, 539193, ..., 404390, 404391, 404392], dtype=int32),
 (539191,))

In [8]:
val_df_inds, val_df_inds.shape

(array([404393, 404394, 404395, ..., 539188, 539189, 539190]), (134798,))

In [11]:
train_df.shape, val_df.shape

((923186, 8), (134784, 8))